In [1]:
import cv2
import mediapipe as mp
import numpy as np


In [2]:
def find_xyz(ind_list, landmark):
    a = landmark[ind_list[0]]
    b = landmark[ind_list[1]]
    c = landmark[ind_list[2]]

    first = [a.x,a.y,a.z]
    mid = [b.x,b.y,b.z]
    end = [c.x,c.y,c.z]
    return first, mid, end

In [3]:
def calculate_angle3D(a,b,c,direction):
    """
    calculate_angle3D is divided by left and right side because this function uses external product
    input : a,b,c -> landmarks with shape [x,y,z,visibility]
            direction -> int -1 or 1
                        -1 means Video(photo) for a person's left side and 1 means Video(photo) for a person's right side
    output : angle between vector ba and bc with range 0~360
    """
    # external product's z value
    external_z = (b[0]-a[0])*(b[1]-c[1]) - (b[1]-a[1])*(b[0]-c[0])

    a = np.array(a) #first
    b = np.array(b) #mid
    c = np.array(c) #end

    ba = b-a
    bc = b-c
    dot_result = np.dot(ba, bc)


    ba_size = np.linalg.norm(ba)
    bc_size = np.linalg.norm(bc)
    radi = np.arccos(dot_result / (ba_size*bc_size))
    angle = np.abs(radi*180.0/np.pi)

    # left side
    if external_z * direction > 0:
        angle = 360 - angle

    return angle

In [ ]:
def calculate_angle2D(a,b,c,direction):
    """
    calculate_angle2D is divided by left and right side because this function uses external product
    input : a,b,c -> landmarks with shape [x,y,z,visibility]
            direction -> int -1 or 1
                        -1 means Video(photo) for a person's left side and 1 means Video(photo) for a person's right side
    output : angle between vector ba and bc with range 0~360
    """
    # external product's z value
    external_z = (b[0]-a[0])*(b[1]-c[1]) - (b[1]-a[1])*(b[0]-c[0])
    
    a = np.array(a[:2]) #first
    b = np.array(b[:2]) #mid
    c = np.array(c[:2]) #end
    
    ba = b-a
    bc = b-c
    dot_result = np.dot(ba, bc)
    
    
    ba_size = np.linalg.norm(ba)
    bc_size = np.linalg.norm(bc)
    radi = np.arccos(dot_result / (ba_size*bc_size))
    angle = np.abs(radi*180.0/np.pi)
    
    if external_z * direction > 0:
        angle = 360 - angle

    return angle

In [5]:
cap = cv2.VideoCapture(0)
mp_drawing = mp.solutions.drawing_utils
pose = mp.solutions.pose
elbow_state = 'up'
Is_start = False

# 팔굽혀펴기 1회 측정 시간
start = 0
end = 0

#우측 관절값
joint_indx = {'right_elbow':[16,14,12],'right_hip':[12,24,26],'right_knee':[24,26,28]}

"""
temp은 팔굽혀펴기 운동을 한번 할떄마다(내려갔다 올라올 때마다) 해당 관절 부분의 각도를 최신화함. 이전에 했던 팔굽혀펴기의 각도값들은 다 초기화함.
      -> 앱 상에서 관절 각도값을 저장하는 데에 무리가 가지않게 list를 초기화시켜줌.
"""
temp = {'right_elbow':[],'right_hip':[],'right_knee':[]}
pushups = []

"""
pushups는 list(list)형식이다. 안쪽 list는 length가 5이며 각각 Is_elbow_up, Is_elbow_down, hip_condition, knee_condition, Is_speed_good을 의미한다.
Is_elbow_up -> int 0 or 1 -> 팔꿈치를 완전히 펴면 1, 완전히 펴지 않으면 0
Is_elbow_down -> int 0 or 1 -> 팔꿈치를 완전히 굽히면 1, 완전히 굽히지 않으면 0
hip_condition -> int 0 or 1 or 2 -> 골반이 정상이면 0, 너무 내려가면 1, 너무 올라가면 2
knee_condition -> int 1 or 0 -> 무릎 각도가 정상범위면 1, 너무 내려가면 0
Is_speed_good -> int 1 or 0 -> 팔굽혀펴기 속도가 적당하면(1회당 1초 이상) 1, 너무 빠르면(1회당 1초 미만) 0 
예를 들어, pushup[0]이 [1,0,1,1,1]이면 첫번째 푸쉬업 동작이 올라올 때 팔꿈치를 완전히 폈고, 내려갈때 완전히 팔을 굽히지는 않았으며, 골반은 과도하게 내려갔고,
무릎각도는 정상범위이며 속도가 너무 빠르지않다는 의미이다.
"""

if cap.isOpened():
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    res=(int(width), int(height))
    fourcc = cv2.VideoWriter_fourcc(*'MP4V') #codec
    out = cv2.VideoWriter('4out353525.mp4', fourcc, 20.0, res)
    frame = None
while True:
    try:
        ret, frame = cap.read()
    except cv2.error:
        continue
    if not ret:
        break

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        landmark = results.pose_landmarks.landmark
        for key,val in joint_indx.items():
            first,mid,end = find_xyz(val, landmark)
            angle = calculate_angle2D(first,mid,end, 1) #각도 계산
            mid = mid[0:2]
            cv2.putText(frame, str(int(angle)),tuple(np.multiply(mid, [width,height]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0),5,cv2.LINE_AA)
            temp[key].append(angle)


        ########
    
    
            

AttributeError: module 'mediapipe.python.solutions.pose' has no attribute 'process'